In [ ]:
import pandas as pd
import numpy as np
import requests
import csv
import folium
import ast

In [ ]:
# Load the data
df = pd.read_excel('MergedData.xlsx')

# List of columns that contain hospital names
columns = ['Providers_08_09', 'Providers_09_10', 'Providers_10_11', 'Providers_11_12', 
           'Providers_12_13', 'Providers_13_14', 'Providers_14_15', 'Providers_15_16', 
           'Providers_16_17', 'Providers_17_18', 'Providers_18_19', 'Providers_19_20', 
           'Providers_20_21', 'Providers_21_22']

# Collect all unique hospital names
hospitalsList = pd.unique(df[columns].values.ravel('K'))

# Print the list of unique hospital names
print(hospitalsList)


In [ ]:
# Remove nan values from the list
hospitalsList = [hospital for hospital in hospitalsList if not pd.isnull(hospital)]

In [ ]:
print(len(hospitalsList))

In [ ]:
hospitalsList

In [ ]:
# Create a new list with only those names that contain 'Hospital', case-insensitive
hospitals_with_word = [name for name in hospitalsList if isinstance(name, str) and 'hospital' in name.lower()]

# Print the new list
print(hospitals_with_word)

In [ ]:
print(len(hospitals_with_word))

In [ ]:
hospitals = hospitalsList[:100]

In [ ]:
def search_place(api_key, query):
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={query}&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry,place_id&key={api_key}&language=en"
    response = requests.get(url)
    data = response.json()
    if 'candidates' in data and len(data['candidates']) > 0:
        place = data['candidates'][0]
        return place
    else:
        return None

api_key = ''

# Open (or create) a CSV file for writing
with open('hospital_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['name', 'address', 'coordinates', 'place_id']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    not_found_counter = 0  # Counter for places not found
    
    for hospital in hospitals:
        query = hospital
        try:
            place = search_place(api_key, query)
            if place:
                print(f"Place: {place['name']}")
                print(f"Address: {place['formatted_address']}")
                print(f"Coordinates: {place['geometry']['location']}")
                print(f"Place ID: {place['place_id']}")

                # Write the place data to the CSV file
                writer.writerow({'name': place['name'], 'address': place['formatted_address'], 'coordinates': place['geometry']['location'], 'place_id': place['place_id']})
            else:
                not_found_counter += 1
                print(f"Could not find place for {query}. Total not found: {not_found_counter}")
        except UnicodeEncodeError:
            print(f"UnicodeEncodeError occurred for {query}. Skipping this entry.")


In [ ]:
not_found_counter

Could not find total 477 instances 

In [ ]:
# Load the CSV file
df = pd.read_csv('hospital_data.csv')

# Convert 'coordinates' from string to dictionary
df['coordinates'] = df['coordinates'].apply(ast.literal_eval)

# Extract 'lat' and 'lng' into separate columns
df['latitude'] = df['coordinates'].apply(lambda x: x['lat'])
df['longitude'] = df['coordinates'].apply(lambda x: x['lng'])

# Select the first 50 rows

# Create a map centered around the first coordinate
m = folium.Map(location=[df['latitude'].iloc[0], df['longitude'].iloc[0]], zoom_start=13)

# Add points to the map
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m)

# Display the map
m

In [ ]:
# Create a new DataFrame with addresses not in the United Kingdom
df_not_uk = df[~df['address'].str.contains('United Kingdom')]

# Select the 'name' and 'address' columns
df_not_uk = df_not_uk[['name', 'address']]

# Convert the DataFrame to a list of dictionaries
names_addresses_not_in_uk = df_not_uk.to_dict('records')

In [ ]:
names_addresses_not_in_uk

In [ ]:
def get_place_details(api_key, place_id):
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=name,rating,formatted_address,permanently_closed&key={api_key}"
    response = requests.get(url)
    data = response.json()
    if 'result' in data:
        place = data['result']
        return place
    else:
        return None

api_key = ''

# Open the CSV file for reading
with open('hospital_data.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        place_id = row['place_id']
        place = get_place_details(api_key, place_id)
        if place:
            print(f"Name: {place['name']}")
            print(f"Address: {place['formatted_address']}")
            if 'rating' in place:
                print(f"Rating: {place['rating']}")
            else:
                print("No rating available for this place.")
            if 'permanently_closed' in place:
                print(f"Permanently Closed: {place['permanently_closed']}")
            else:
                print("This place is not permanently closed.")
        else:
            print(f"Could not find place for place_id {place_id}.")

